In [1]:
import json
import os
import pandas as pd
from pandas.io.json import json_normalize    


In [37]:
def is_event(record, e):
    return record['type'] == e

In [41]:
class EventParser:
    def __init__(self, data, idx):
        self.raw_data = data
        self.idx = idx
        
    def parse(self, data):
        """Extract attributes from data."""
        NotImplementedError()
        
    def to_dict(self):
        """Attributes to dict."""
        NotImplementedError()
        
class PushEventParser(EventParser):
    def __init__(self, data, idx):
        super(PushEventParser, self).__init__(data, idx)
        self.type = "PushEvent"
        
    def parse(self):
        """Extract attributes from data."""
        self.repo_id = data['repo']['id']
        self.repo_name = data['repo']['name']
        self.before = data['payload']['before']
        self.commits = data['payload']['commits']
        self.processed_commits = self._extract_commits()
        self.distinct = data['payload']['distinct_size']  # Number of commits in payload
        
    def to_dict(self):
        """Attributes to dict list of dicts.
        
        Note that we return list of dicts to handle payloads with lists.
        
        Expect to accumulate data in lists using .extend()
        
        """
        data = []
        d_global = {
            "repo_id": self.repo_id,
            "repo_name": self.repo_name,
            "distinct": self.distinct
        }
        for commit in self.processed_commits:
            commit.update(d_global)
            data.append(commit)
        return data
        
    def _extract_commits(self):
        processed_commits = []
        for i, commit in enumerate(self.commits):
            curr_commmit = {
                'commit_idx': i,
                'commit_sha': commit['sha'],
                'is_unique': commit['distinct'],
                'message': commit['message']
            }
            processed_commits.append(curr_commmit)
        return processed_commits

In [43]:
limit = 3
i = 0
d = []
with open("../practice_gh.json", 'r') as f:
    for i, record in enumerate(f):
        data = json.loads(record)
        if is_event(data, 'PushEvent'):
            parser = PushEventParser(data, i)
            parser.parse()
            curr_data = parser.to_dict()
            d.extend(curr_data)

In [45]:
df = pd.DataFrame(d)

In [49]:
# df.sort_values('distinct', ascending=False)
df[df['repo_id'] == 15841152]

,commit_idx,commit_sha,distinct,is_unique,message,repo_id,repo_name
2547,0,568cfcc7ba58e46ab8e282ba87a4fc5a01b37365,2287,True,lwt 2.4.6,15841152,pippijn/opam-repository
2548,1,af9f482a6cac522d5df6754a2186c50fba8eef2c,2287,True,tighter constraint on rss for stog 0.4 -> 0.7,15841152,pippijn/opam-repository
2549,2,a6444b805dc3489f1c08ea0aff2a02b277920633,2287,True,Merge branch 'master' of https://github.com/OC...,15841152,pippijn/opam-repository
2550,3,8a005b284fc80e5af176a8c14c77364c5b5103ce,2287,True,OCaml version specified,15841152,pippijn/opam-repository
2551,4,656fd3e9f93d8e1e4223fc225d53c478568973f3,2287,True,-j jobs option added to Coq IDE,15841152,pippijn/opam-repository
2552,5,da48845f05839ea33460ec288bc4df00c07320a2,2287,True,Older packages updated against OCaml 4.02.0,15841152,pippijn/opam-repository
2553,6,93e9e8b02a6b0f7ffe37f7b871ec6c7390b1bacf,2287,True,Merge pull request #2846 from zoggy/master\n\n...,15841152,pippijn/opam-repository
2554,7,d4510a0e8e24ec884bf554f445ec96d10d061dc2,2287,True,add camlp4 4.02.0+2 and camlp4 4.02.1,15841152,pippijn/opam-repository
2555,8,d60bd9255f0e13dc93fe07c7ad3e6db25a6a3050,2287,True,typo,15841152,pippijn/opam-repository
2556,9,4c0bc926b45e04ce3bdb81da37e00d353250a78c,2287,True,Add 4.02.1 compiler switches,15841152,pippijn/opam-repository


In [47]:
df.groupby(by=['repo_id']) \
    .count() \
    .reset_index(drop=False) \
    .sort_values('repo_name', ascending=False)

,repo_id,idx,repo_name
2488,21481110,80,80
135,702550,64,64
5066,28648149,42,42
1713,15233168,36,36
5886,28689169,30,30
786,6058234,30,30
3105,25173910,29,29
3546,26730195,26,26
5496,28688327,24,24
2577,22066165,24,24
